## 감성 분석(sentiment analysis)

                                                    note by D.H.Yeom
                                                    
* 감성 분석: 문서에 대해 좋다(positive) 혹은 나쁘다(negative)로 평가

#1.라이브러리(Library) & 데이터(Data)

In [ ]:
!pip install konlpy

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}', format(device_name))

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request

from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

* 데이터 로드



In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
#urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
#urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

### Drescrition
* id, document, label로 구성.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
print('학습용 데이터 리뷰 개수 :',len(train_data))

In [ ]:
# 학습용 데이터 리뷰 개수
train_data.shape

In [ ]:
# 상위 5개 출력
train_data[:5]

In [ ]:
# test_data의 리뷰 개수 확인
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

In [ ]:
test_data[:5]

#2. 데이터 전처리

* # train_data의 데이터 중복 유무 확인

In [ ]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

**[결과 해석]**
* 총 150,000개의 샘플 가운데 document 중복을 제거한 샘플 개수가 146,182개로 약 4,000개의 중복 샘플이 존재.
* label 열은 0 또는 1의 두 가지 값만을 가지므로 '2' 출력.

* 중복 샘플 제거

In [ ]:
# document 열의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
#중복제거 확인
print('샘플수  :',len(train_data))

* train_data에서 해당 리뷰의 긍, 부정 유무가 기재되어있는 레이블(label) 값의 분포 확인.

In [ ]:
train_data['label'].value_counts().plot(kind = 'bar')

* 샘플의 개수 확인

In [ ]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

**[결과확인]**
* 레이블이 0인 리뷰가 근소하게 많음.

### 리뷰 중에 Null 값을 가진 샘플 여부 확인

In [ ]:
print(train_data.isnull().values.any())

**[결과확인]**
* True : 데이터 중에 Null 이 존재함.
* 어떤 열에 존재하는지 확인.

In [ ]:
train_data.loc[train_data.document.isnull()]

* Null 값을 가진 샘플 제거

In [ ]:
train_data = train_data.dropna(how = 'any')    # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any())        # Null 값이 존재하는지 확인

Null 샘플이 제거, 제거여부 확인

In [ ]:
print(len(train_data))

### 정규분포식 적용

**[한글에 적용]**
* 먼저 자음과 모음에 대한 범위 지정.
* 일반적으로 자음의 범위는 ㄱ ~ ㅎ, 모음의 범위는 ㅏ ~ ㅣ와 같이 지정 가능
* 해당 범위 내의 자음과 모음은  아래의 링크 참조.
* 링크 : https://www.unicode.org/charts/PDF/U3130.pdf
ㄱ ~ ㅎ: 3131 ~ 314E
ㅏ ~ ㅣ: 314F ~ 3163
* 완성형 한글의 범위는 가 ~ 힣과 같이 사용
* 해당 범위 내에 포함된 음절들은 아래의 링크에서 확인
* 링크 : https://www.unicode.org/charts/PDF/UAC00.pdf

**[영어 예시]**
* 알파벳들을 나타내는 정규 표현식은 [a-zA-Z].
* 영어의 소문자와 대문자들을 모두 포함하고 있는 정규 표현식으로, 영어에 속하지 않는 구두점이나 특수문자를 제거할 수 있음.

In [ ]:
# 알파벳과 공백을 제외하고 모두 제거
eng_text = 'Do you really think so? Well, I think we need to talk about it... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
print(re.sub(r'[^a-zA-Z ]', '', eng_text))

In [ ]:
train_data[:5]

* train_data 한글만 남기고 제거.

In [ ]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

**[정리]**
* 기존의 공백(띄어쓰기)은 유지되면서 구두점 등은 제거
* 영화 리뷰는 한글이 아니더라도 영어, 숫자, 특수문자로도 리뷰를 업로드할 수 있음. 즉, 기존에 한글이 없는 리뷰였다면 아무런 값이 없는 빈(empty) 값이 되었을 것임.
* train_data에 공백(whitespace)만 있거나 빈 값을 가진 행이 있다면 Null 값으로 변경하도록 하고, Null 값이 존재하는지 확인 필요.

In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "")      # white space 데이터를 빈값(empty value)로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

**[결과확인]**
* Null 값이 789개 확인됨.

In [ ]:
# Null 값의 행 출력
train_data.loc[train_data.document.isnull()][:5]

* Null 샘플은 리뷰가 없으믈로 긍,부정 데이터에 의미가 없음
* 따라서 데이터를 제거해야 함

In [ ]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

### 테스트 데이터도 동일하게 전처리 과정 수행

In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True)                      # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "")                # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True)                             # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('샘플의 개수 :',len(test_data))

#3. 토큰화
* 불용어 제거: 불용어는 한국어의 조사, 접속사 등의 보편적인 불용어를 사용할 수도 있으나, 데이터를 지속적으로 검토하면서 추가하는 경우도 많음.

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

**[형태소 분석기 적용]**
* 형태소 분석기는 KoNLPy의 Okt 사용

In [ ]:
# 예시
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔. 흥행 실패는 따논 당상이군', stem = True)

* stem = True를 사용하면 일정 수준의 정규화를 수행해 줌.
* '이런'이 '이렇다'로, '만드는'이 '만들다'로 변환.

* train_data에 형태소 분석기를 사용하여 토큰화를 하면서 불용어 제거.
* 결과를 X_train에 저장.

In [ ]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)                                            # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]     # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
# 결과 확인
print(X_train[:3])

* 테스트 데이터에 대해서도 동일하게 토큰화 실행

In [ ]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)                                        # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

#4. 정수 인코딩
* 텍스트를 숫자로 처리할 수 있도록 학습 데이터와 테스트 데이터에 정수 인코딩 수행.
* 먼저 학습 데이터에 대해 단어 집합(vocaburary)을 만든다.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

* 단어 집합이 생성되고, 동시에 각 단어에 고유한 정수가 부여되었음
* tokenizer.word_index를 출력하여 확인 가능.

In [ ]:
print(tokenizer.word_index)

* 높은 정수가 부여된 단어들은 등장 빈도수가 매우 낮다
* 빈도수가 낮은 단어들은 제거.
* 빈도수가 3회 미만인 단어의 비중 확인.

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index)        # 단어의 수
rare_cnt = 0                                 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0                               # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0                                # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

* 등장 빈도가 threshold 값인 3회 미만은 단어 집합에서 절반 이상
* 실제로 훈련 데이터에서 등장 빈도로 차지하는 비중은 상대적으로 매우 적은 1.87%
* 등장 빈도가 2회 이하인 단어들은 자연어 처리에서 별로 중요하지 않음
* 이 단어들은 정수 인코딩 과정에서 배제.

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

* 단어 집합의 크기는 19,416개
* 이를 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환.

In [ ]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

* 정수 인코딩 진행 확인: X_train에 대해서 상위 3개의 샘플 출력

In [ ]:
print(X_train[:3])

**[정리]**
* 각 샘플 내의 단어들은 각 단어에 대한 정수로 변환된 것을 확인
* 단어의 개수는 19,416개로 제한되었으므로 0번 단어 ~ 19,415번 단어까지만 사용
* 0번 단어는 패딩을 위한 토큰임에 주의.
* train_data에서 y_train과 y_test를 별도로 저장.

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

**[빈 샘플(empty samples)]제거**
* 전체 데이터에서 빈도수가 낮은 단어가 삭제되었다는 것은 빈도수가 낮은 단어만으로 구성되었던 샘플들은 빈(empty) 샘플이 되었다는 것을 의미
* 빈 샘플들은 어떤 레이블이든 의미가 없으므로 빈 샘플들을 제거해야 함
* 각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스를 확인.

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

**[정리]**
* drop_train에는 X_train으로부터 얻은 빈 샘플들의 인덱스가 저장됨.
* 앞서 학습 데이터(X_train, y_train)의 샘플 개수는 145,791개임을 확인
* 빈 샘플들을 제거한 후의 샘플 개수 확인 필요

In [ ]:
# 빈 샘플 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

**[정리]**
* 145,162개로 샘플의 수가 줄어든 것을 확인

**[패딩 실행]**
* 서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 작업
* 전체 데이터에서 가장 길이가 긴 리뷰와 전체 데이터의 길이 분포 확인

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

* 가장 긴 리뷰의 길이는 69
* 그래프를 봤을 때 전체 데이터의 길이 분포는 대체적으로 약 11내외
* 모델이 처리할 수 있도록 X_train과 X_test의 모든 샘플의 길이를 특정 길이로 동일하게 맞춰줄 필요가 있음
* 특정 길이 변수를 max_len으로 정함.
* 대부분의 리뷰가 내용이 잘리지 않도록 할 수 있는 최적의 max_len의 값 확인
* 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수를 만듬.

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

* 위의 분포 그래프에서, max_len = 30이 적당할 것으로 판단.
* 이 값이 얼마나 많은 리뷰 길이를 커버하는지 확인

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

* 전체 훈련 데이터 중 약 94%의 리뷰가 30이하의 길이를 가지는 것을 확인
* 따라서, 모든 샘플의 길이를 30으로 맞춤.

#-------
##굿럭!!!

* codecs 패키지: 유니코드로 인코딩하며 읽기 위해
* 읽어들인 결과는 유니코드 문자열

In [ ]:
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]                             # header 제외

* 데이터가 id, 내용, 평점으로 구성.내용을 X, 평점을 y로 저장.
* zip(*시퀀스데이터) 함수는 시퀀스형 데이터 타입을 인수로 하여 각 element들이 순서에 맞게 쌍으로 생로운 zip이라는 데이터 타입 object를 생성하는 내장함수이다.
* zip()된 결과를 다시 원래의 시퀀스데이터로 만들고자 할때 zip(*시퀀스데이터)을 하여 unpacking함.

In [ ]:
X = list(zip(*data))[1]
y = np.array(list(zip(*data))[2], dtype=int)

 * 다항 나이브 베이즈 모형으로 학습

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #각 문서에 어떤 단어가 몇 번 등장했는지를 파악할 때 사용합니다. 단어를 세서(count) 문서를 벡터화(vectorize)한다는 의미입니다
from sklearn.naive_bayes import MultinomialNB    #naive bayes 분류기, 클래스별로 특성의 평균을 계산. 카운트 데이터(ex 문장에 나타난 단어의 횟수)에 적용가능
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report  #분류 모델의 평가 지표를 출력해주는 함수

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

* 모형의 성능 확인을 위해 테스트 데이터 로드

In [ ]:
%%time
model1.fit(X, y)

In [ ]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
    data_test = [line.split('\t') for line in f.read().splitlines()]
    data_test = data_test[1:]   # header 제외

In [ ]:
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

* 분류 모델의 평가 지표: 정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1-score.
* Precision(정밀도): 예측한 클래스 중 실제로 해당 클래스인 데이터의 비율.
* Recall(재현율): 실제 클래스 중 예측한 클래스와 일치한 데이터의 비율.
* F1-score: Precision과 Recall의 조화평균.
* Support: 각 클래스의 실제 데이터 수.

* Tfidf 방법을 사용한 결과와 비교

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [ ]:
%%time
model2.fit(X, y)

In [ ]:
print(classification_report(y_test, model2.predict(X_test)))

* 형태소 분석기를 사용한 결과와 비

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
pos_tagger = Okt()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [ ]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB()),
])

In [ ]:
%%time
model3.fit(X, y)

In [ ]:
print(classification_report(y_test, model3.predict(X_test)))

* gram 을 사용하여 성능 개선
- N-gram은 문자열에서 N개의 연속된 요소를 추출하는 방법.

In [ ]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

In [ ]:
%%time
model4.fit(X, y)

In [ ]:
print(classification_report(y_test, model4.predict(X_test)))